In [709]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [707]:
# 타이타닉 데이터 다운로드
!kaggle competitions download -c titanic

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [708]:
## 데이터 분석 관련
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 데이터 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') # matplotlib의 스타일에 관련한 함
## 그래프 출력에 필요한 IPython 명령어
%matplotlib inline 

In [710]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
gender_submission = pd.read_csv("gender_submission.csv")

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [711]:
train_df.info()
print('-'*28)
test_df.info()
# train의 attribute는 12개 test의 attribute는 11개인 이유는 train은 Survied의 여부를 알고 있기 때문이다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-

###**여기서 주의 깊게 봐야할 부분은 다음과 같다.**
+ 각 데이터는 빈 부분이 있는가?
  + 빈 부분이 있다면, drop할 것인가 아니면 default값으로 채워 넣을 것인가.
  + cabin, Age, Embarked 세 항목에 주의
+ 데이터는 float64로 변환할 수 있는가. 
  + 아니라면 범주형 데이터로 만들 수 있는가.

In [712]:
# 이름과 티켓에서 가져올 수 있는 데이터는 없기 때문에 PassengerID와 이름, 티켓을 지운다. 
# 하지만 이 문제에서 결과물은 'PassengerId', 'Survived' 요소가 필요하므로 훈련데이터에서만 삭제한다.
train_df = train_df.drop(['PassengerId', 'Ticket'], axis=1) # axis = 1은 열을 지운다. 
test_df = test_df.drop(['Ticket'], axis=1) # 결과물은 test에서 나온다. 즉, PassengerId를 지우면 안된다.

###**데이터 하나하나 처리하기**
이제 남은 데이터 종류는 다음과 같다.
1. Pclass
2. Sex
3. SibSp
4. Parch
5. Fare
6. Cabin
7. Embarked
8. Name
9. Age(추가)

In [713]:
# # 1. Pclass
# # 서수형 데이터이다. 1등석, 2등석, 3등석과 같은 정보. 처음에 확인시에 데이터가 비어있지 않은 것을 확인할 수 있었다.
# # 데이터에 대한 확인과 데이터를 변환해보도록 하겠다. 우선 각 unique한 value에 대한 카운팅은 value_counts() 메서드로 확인할 수 있다.
# train_df['Pclass'].value_counts()

# # 1, 2, 3은 정수이니, 그냥 실수로만 바꾸면 되지않을까 생각할 수 있다. 하지만 1, 2, 3 등급은 경우에 따라 다를 수 있지만 연속적인 정보가 아니며, 각 차이 또한 균등하지 않다.
# # 그렇기에 범주형(카테고리) 데이터로 인식하고 인코딩해야한다.(비슷한 예시로 영화 별점 등이 있다.)
# # 이 데이터는 범주형 데이터이므로 one-hot-encoding을 pd.get_dummies() 메서드로 인코딩한다.
# pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
# pclass_test_dummies = pd.get_dummies(test_df['Pclass'])

# train_df.drop(['Pclass'], axis=1, inplace=True)
# test_df.drop(['Pclass'], axis=1, inplace=True)

# train_df = train_df.join(pclass_train_dummies)
# test_df = test_df.join(pclass_test_dummies)

In [714]:
train_df.head() # 원래는 columns의 이름을 설정하고, 넣어줘야하는데 실수로 넣지 않아 1, 2, 3 이라는 컬럼으로 데이터가 들어갔다.

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S


In [715]:
# 2. Sex
# 성별이라는 뜻으로 남과 여로 나뉘므로 이 또한 one-hot-encoding을 진행
# # [[[ dummie 방식 ]]]
# sex_train_dummies = pd.get_dummies(train_df['Sex'])
# sex_test_dummies = pd.get_dummies(test_df['Sex'])

# train_df.drop(['Sex'], axis=1, inplace=True)
# test_df.drop(['Sex'], axis=1, inplace=True)

# train_df = train_df.join(sex_train_dummies)
# test_df = test_df.join(sex_test_dummies)

# category .cat.codes방식 numeric
train_df['Sex'] = train_df['Sex'].astype('category').cat.codes
test_df['Sex'] = test_df['Sex'].astype('category').cat.codes
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",1,35.0,0,0,8.0500,NaN,S


In [716]:
# 3,4. SibSp & Parch 
# 형제 자매와 부모님은 가족으로 함께 처리할 수 있다. 하지만 굳이 바꿀필요는 없다.
train_df['Family'] = 1 + train_df['SibSp'] + train_df['Parch']
test_df['Family'] = 1 + train_df['SibSp'] + test_df['Parch']

train_df = train_df.drop(['SibSp', 'Parch'], axis=1)
test_df = test_df.drop(['SibSp', 'Parch'], axis=1)

# + Solo : 내가 혼자 탔는지 다른 가족과 탔는지 여부를 구분해주는 데이터 추가
train_df['Solo'] = (train_df['Family'] == 1)
test_df['Solo'] = (test_df['Family'] == 1)

# 5. Fare
# 탑승료이다. 신기하게 test 데이터셋에 1개의 데이터가 비어있다. 아마 디카프리오인듯 하다. 우선 빈 부분을 fillna 메서드로 채운다.
# 데이터 누락이 아닌 무단 탑승이라 생각하고 0으로 입력
test_df['Fare'].fillna(0, inplace=True)
train_df.head()

,Survived,Pclass,Name,Sex,Age,Fare,Cabin,Embarked,Family,Solo
0,0,3,"Braund, Mr. Owen Harris",1,22.0,7.2500,NaN,S,2,False
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,71.2833,C85,C,2,False
2,1,3,"Heikkinen, Miss. Laina",0,26.0,7.9250,NaN,S,1,True
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,53.1000,C123,S,2,False
4,0,3,"Allen, Mr. William Henry",1,35.0,8.0500,NaN,S,1,True


In [717]:
# 6. Cabin
# 객실이라는 뜻이다. 대부분이 NaN이므로 버린다.
train_df = train_df.drop(['Cabin'],axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [718]:
# 7. Embarked
# 탑승 항구를 의미, 우선 데이터를 확인
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [719]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [720]:
# S가 대다수이고 일부 데이터가 비어있는 것을 알 수 있다. 빈 부분은 S로 우선 채운다(.info로 확인했을 때 빈 부분이 있는 줄 몰랐다)
train_df["Embarked"].fillna('S', inplace=True)

# Embarked 컬럼 역시 numeric 한 데이터로 변경
train_df['Embarked'] = train_df['Embarked'].astype('category').cat.codes
test_df['Embarked'] = test_df['Embarked'].astype('category').cat.codes
train_df.head()

,Survived,Pclass,Name,Sex,Age,Fare,Embarked,Family,Solo
0,0,3,"Braund, Mr. Owen Harris",1,22.0,7.2500,2,2,False
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,71.2833,0,2,False
2,1,3,"Heikkinen, Miss. Laina",0,26.0,7.9250,2,1,True
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,53.1000,2,2,False
4,0,3,"Allen, Mr. William Henry",1,35.0,8.0500,2,1,True


In [721]:
# 8. Name
# Title은 'Name' 칼럼에서 ~씨와 같은 t itle을 추출하여 새롭게 생성해주는 컬럼,
# 단 주의해야할 점은 TItle을 추출하여 카테고리와 해주면, 데이터의 총량 비하여 너무 복잡도가 올라가는 경향이있다.
# 그렇기에 모수가 적은 Mlle, Mme, Ms는 단일화 시켜주어야 한다.
train_df['Title'] = train_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_df['Title'] = test_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
test_df['Title'] = test_df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')

train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

train_df['Title'].value_counts()

Mr        517
Miss      182
Mrs       125
Master     40
Other      23
Mlle        2
Mme         1
Ms          1
Name: Title, dtype: int64

In [722]:
# 위에서 세어보니 Mlle, Mme, Ms의 수가 적어서 단일화 해주는 작업을 해야한다.
train_df['Title'] = train_df['Title'].replace(['Mlle', 'Ms'], 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

test_df['Title'] = test_df['Title'].replace(['Mlle', 'Ms'], 'Miss')
test_df['Title'] = test_df['Title'].replace('Mme', 'Mrs')

train_df['Title'] = train_df['Title'].astype('category').cat.codes
test_df['Title'] = test_df['Title'].astype('category').cat.codes

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Family,Solo,Title
0,0,3,1,22.0,7.2500,2,2,False,2
1,1,1,0,38.0,71.2833,0,2,False,3
2,1,3,0,26.0,7.9250,2,1,True,1
3,1,1,0,35.0,53.1000,2,2,False,3
4,0,3,1,35.0,8.0500,2,1,True,2


In [723]:
# 9. Age
# 나이는 연속형 데이터이므로, 큰 처리가 필요없다. (카테고리화를 하여 일부 알고리즘에 더 유용한 결과를 만들 수 있다.)
# 하지만 일부 NaN 데이터가 있으니 이를 채울 수 있는 방법에 대해서 생각해보자
# 1. 랜덤(random), 2. 평균값(mean), 3. 중간값(median), 4. 데이터 버리기(drop)

# groupyby 함수를 이용해 "Title" 컬럼의 그룹을 나누어(1, 2, 3) 해당 그룹의 "Age"칼럼의 median을 fillna에 대입
train_df["Age"].fillna(train_df.groupby("Title")["Age"].transform("median"), inplace=True)
test_df["Age"].fillna(test_df.groupby("Title")["Age"].transform("median"), inplace=True)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Family,Solo,Title
0,0,3,1,22.0,7.2500,2,2,False,2
1,1,1,0,38.0,71.2833,0,2,False,3
2,1,3,0,26.0,7.9250,2,1,True,1
3,1,1,0,35.0,53.1000,2,2,False,3
4,0,3,1,35.0,8.0500,2,1,True,2


In [724]:
# Age를 구간화 (Binning), 5세 단위로 자르고 50대는 10세단위 그리고 60세 이상은 모두 묶어서 Binning해줌
# Train
train_df.loc[ train_df['Age'] <= 10, 'Age'] = 0
train_df.loc[(train_df['Age'] > 10) & (train_df['Age'] <= 16), 'Age'] = 1
train_df.loc[(train_df['Age'] > 16) & (train_df['Age'] <= 20), 'Age'] = 2
train_df.loc[(train_df['Age'] > 20) & (train_df['Age'] <= 26), 'Age'] = 3
train_df.loc[(train_df['Age'] > 26) & (train_df['Age'] <= 30), 'Age'] = 4
train_df.loc[(train_df['Age'] > 30) & (train_df['Age'] <= 36), 'Age'] = 5
train_df.loc[(train_df['Age'] > 36) & (train_df['Age'] <= 40), 'Age'] = 6
train_df.loc[(train_df['Age'] > 40) & (train_df['Age'] <= 46), 'Age'] = 7
train_df.loc[(train_df['Age'] > 46) & (train_df['Age'] <= 50), 'Age'] = 8
train_df.loc[(train_df['Age'] > 50) & (train_df['Age'] <= 60), 'Age'] = 9
train_df.loc[ train_df['Age'] > 60, 'Age'] = 10

# Test
test_df.loc[ test_df['Age'] <= 10, 'Age'] = 0
test_df.loc[(test_df['Age'] > 10) & (test_df['Age'] <= 16), 'Age'] = 1
test_df.loc[(test_df['Age'] > 16) & (test_df['Age'] <= 20), 'Age'] = 2
test_df.loc[(test_df['Age'] > 20) & (test_df['Age'] <= 26), 'Age'] = 3
test_df.loc[(test_df['Age'] > 26) & (test_df['Age'] <= 30), 'Age'] = 4
test_df.loc[(test_df['Age'] > 30) & (test_df['Age'] <= 36), 'Age'] = 5
test_df.loc[(test_df['Age'] > 36) & (test_df['Age'] <= 40), 'Age'] = 6
test_df.loc[(test_df['Age'] > 40) & (test_df['Age'] <= 46), 'Age'] = 7
test_df.loc[(test_df['Age'] > 46) & (test_df['Age'] <= 50), 'Age'] = 8
test_df.loc[(test_df['Age'] > 50) & (test_df['Age'] <= 60), 'Age'] = 9
test_df.loc[ test_df['Age'] > 60, 'Age'] = 10

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Family,Solo,Title
0,0,3,1,3.0,7.2500,2,2,False,2
1,1,1,0,6.0,71.2833,0,2,False,3
2,1,3,0,3.0,7.9250,2,1,True,1
3,1,1,0,5.0,53.1000,2,2,False,3
4,0,3,1,5.0,8.0500,2,1,True,2


### Feature와 Label을 정의하기

In [725]:
feature = [
           'Pclass',
           'Sex',
           'Age',
           'Fare',
           'Embarked',
           'Family',
           'Solo',
           "Title",
]
label = [
         'Survived',
]

### HyperParameter
여러 가지 모델링을 해본 결과, 이 블로그에서 진행항 pre-processing 데이터셋에는 RnadomForestClassifier가 가장 좋은 결과를 가져다 주었다.

---

우선, 이번 Titanic 생존자 예측 대회에서는 dataset의 복잡도가 크지 않고, 사이즈도 매우 적기 때문에 n_estimator 값은 최대한 줄이는 전략을 취했다. 
또한 max_depth도 제한을 두어 너무 깊어지지 않도록 했으며, 다른 parameter는 별도로 건들이지 않았다.

In [740]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

data = train_df[feature]
target = train_df[label]

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

clf = RandomForestClassifier(n_estimators=50, max_depth=6, random_state=0)
cross_val_score(clf, data, target, cv=k_fold, scoring='accuracy', ).mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

0.8305493133583021

In [739]:
train_x = train_df[feature]
train_y = train_df[label]
test_x = test_df[feature]

clf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=0)
clf.fit(train_x, train_y)
gender_submission['Survived'] = clf.predict(test_x)
gender_submission.to_csv('titanic-submission.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###**여러 머신러닝 알고리즘 적용 해보기**

In [741]:
## Scikit-Learn의 다양한 머신러닝 모듈을 불러옵니다.
## 분류 알고리즘 중에서 선형회귀, 서포트벡터머신, 랜덤포레스트, K-최근접이웃 알고리즘을 사용해보려고 한다.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Logistic Regression
logreg = LogisticRegression(max_iter=1000) # max_iter을 이용하여 오류해결
logreg.fit(train_x, train_y)

pred_y = logreg.predict(test_x)
logreg.score(train_x, train_y)

# Support Vector Machines
svc = SVC()
svc.fit(train_x, train_y)

pred_y = svc.predict(test_x)
svc.score(train_x, train_y)

# Random Forests
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(train_x, train_y)

pred_y = random_forest.predict(test_x)
random_forest.score(train_x, train_y)

# K-Neigbor
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(train_x, train_y)

pred_y = knn.predict(test_x)
knn.score(train_x, train_y)

# Random Forests
random_forest = RandomForestClassifier(n_estimators=1)
random_forest.fit(train_x, train_y)
pred_y = random_forest.predict(test_x)
random_forest.score(train_x, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8002244668911336